# Energy Out-of-distribution detection

From the paper [Energy-based Out-of-distribution Detection](http://arxiv.org/abs/2010.03759). Let's explore the folder structure, the models and the available scripts and extract what we need for our ideas.

In [1]:
import torch
from torchvision import datasets, transforms

from anomaly_scores.energy import energy_anomaly_score
from anomaly_scores.softmax import max_softmax_anomaly_score
from energy_ood.CIFAR.models.wrn import WideResNet
from energy_ood.utils.display_results import show_performance
from energy_ood.utils.svhn_loader import SVHN
from util import TEST_TRANSFORM
from util.get_ood_score import get_ood_scores

## Model

We are using the Wide ResNet as in the paper.

In [2]:
model = WideResNet(depth=40, num_classes=10, widen_factor=2, dropRate=0.3)
model.load_state_dict(
    torch.load(
        "energy_ood/CIFAR/snapshots/pretrained/cifar10_wrn_pretrained_epoch_99.pt",
    )
)
model.eval()
_ = model.cuda()

## The data
Let's start with replicating the results from the paper. First, with the SVHN data set. 

In [3]:
loaders = []

id_data = datasets.CIFAR10("data/cifar10", train=False, transform=TEST_TRANSFORM)
id_loader = torch.utils.data.DataLoader(
    id_data, batch_size=200, shuffle=False, num_workers=2, pin_memory=True
)
loaders.append(("CIFAR10", id_loader))


ood_data = SVHN(
    root="data/svhn/",
    split="test",
    transform=TEST_TRANSFORM,
    download=False,
)
ood_loader = torch.utils.data.DataLoader(
    ood_data, batch_size=200, shuffle=True, num_workers=2, pin_memory=True
)
ood_num_examples = len(loaders[0][1].dataset) // 5
loaders.append(("SVHN", ood_loader))

# Max Softmax anomaly score

In [4]:
in_score = get_ood_scores(id_loader, model, max_softmax_anomaly_score, ood_num_examples, in_dist=True)
out_score = get_ood_scores(ood_loader, model, max_softmax_anomaly_score, ood_num_examples)
show_performance(out_score, in_score, method_name="Max Softmax")

			Max Softmax
FPR95:			31.25
AUROC:			92.08
AUPR:			90.51


Can only reproduce the score of the paper when switching out and in. Is this reasonable?

In [5]:
show_performance(-in_score, -out_score, method_name="Max Softmax")

			Max Softmax
FPR95:			47.45
AUROC:			92.08
AUPR:			93.07


# Energy anomaly score

In [6]:
in_score = get_ood_scores(
    id_loader, model, energy_anomaly_score, ood_num_examples, in_dist=True
)
out_score = get_ood_scores(ood_loader, model, energy_anomaly_score, ood_num_examples)
show_performance(out_score, in_score, method_name="Energy")

			Energy
FPR95:			45.50
AUROC:			90.83
AUPR:			90.79


And same here. Works only when switching the field?

In [7]:
show_performance(-in_score, -out_score, method_name="Energy")

			Energy
FPR95:			34.80
AUROC:			90.83
AUPR:			89.97
